# EXAMPLES (TQL)
- [Tensor Query Language (TQL)](https://docs.activeloop.ai/examples/tql)
  - [TQL Syntax](https://docs.activeloop.ai/examples/tql/syntax)
  - [**Index for ANN Search**](https://docs.activeloop.ai/examples/tql/ann-index)
    - [**Caching and Optimization**](https://docs.activeloop.ai/examples/tql/ann-index/caching-and-optimization)
  - [Sampling Datasets](https://docs.activeloop.ai/examples/tql/sampling)

## Index for ANN Search

### Deep Lake Implements an Index for ANN Search
*Deep Lake implements the Hierarchical Navigable Small World (HSNW) index for Approximate Nearest Neighbor (ANN) search. The index is based on the OSS Hsnwlib package with added optimizations. The implementation enables users to run queries on >35M embeddings in less than 1 second.*

#### Unique aspects of Deep Lake's HSNW implementation
- *Rapid index creation with multi-threading optimized for Deep Lake*
- *Efficient memory management that reduces RAM usage*

#### Memory Management in Deep Lake
**RAM Cost  >>  On-disk Cost  >>  Object Storage Cost**<br>
*Minimizing RAM usage and maximizing object store significantly reduces costs of running a Vector Database. Deep Lake has a unique implementation of memory allocation that minimizes RAM requirement without any performance penalty:*<br>
[Memory Architecture for the Deep Lake Vector Store](https://docs.activeloop.ai/~gitbook/image?url=https%3A%2F%2Fcontent.gitbook.com%2Fcontent%2FWOs95B2h3lcO4dwXDRJ3%2Fblobs%2Fa8UIj0ohjpnKm98rkz8X%2FIndex_Memory_Usage.png&width=400&dpr=3&quality=100&sign=4e7045b9&sv=2)

#### Using the Index
*By default, Deep Lake performs linear embedding search for up to 100,000 rows of data. Once the data exceeds that limit, the index is created and the embedding search uses ANN.  This index threshold is chosen because linear search is the most accurate, and for less than 100,000 rows, there is almost no performance penalty compared to ANN search.*

*You may update the threshold for creating the index using the API below:*
```
vectorstore = VectorStore(path, index_params = {threshold: <threshold_int>})
```

#### Limitations
*The following limitations of the index are being implemented in upcoming releases:*<br>
*If the search is performed using a combination of attribute and vector search, the index is not used and linear search is applied instead.*

## Caching and Optimization

### Extract Maximum Performance from Your Vector Search

#### Tuning the Index Parameters

*The parameters of the HSNW index can be tuned using the index_params shown below:*
```
vectorstore = VectorStore(path, 
                          index_params = {"threshold": -1,
                                          "distance_metric":"COS",
                                          "additional_params": {
                                              "efConstruction": 600,
                                              "M": 32}})
```

#### Caching of Embeddings and Index

*Either of the following operations caches the embeddings are on disk and the index in RAM:*<br>

- *The index is created*
- *The first vector search is executed after the Vector Store is loaded*<br>

*Since the first query caches critical information, subsequent queries will execute much faster compared to the first query. Since the cache is invalidated after the Vector Store is loaded or initialized, the optimal access pattern is not to re-load the Vector Store each search, unless you believe it was updated by another client.*

*The embeddings are cached on disk in the following locations:*
```
Mac: /tmp/....
Linux: /var/folders/
```

#### Caching of Other Tensors

*Tensors containing other information such as text and metadata are also cached in memory when they are used in queries. As a result, the first query that utilized this data will be slowest, with subsequent queries running much faster.*<br> 

*If the data size exceeds the cache size, it will be re-fetched with every query, thus reducing query performance. The default cache size is 2 MB, and you may increase the cache size using the parameter below:*

```
vectorstore = VectorStore(path, memory_cache_size = <cache_in_MB))
```